In [1]:
import pandas as pd
import numpy as np
import scipy.spatial
from scipy.optimize import minimize

In [2]:
hh="Python_HH_Demo_Data.csv"
hh_df = pd.read_csv(hh)
mk="MCCORMICK_ASSIGNMENT.csv"
mk_df = pd.read_csv(mk)
panelfile="Panel_Demo.csv"
panel_df=pd.read_csv(panelfile)

In [3]:
hh_df_new=pd.crosstab(hh_df["MOSAIC_SEGMENT_ID"],hh_df["Value"],values=hh_df["Count"],aggfunc=np.sum)


In [4]:
hh_df_member=hh_df_new.drop(['African American','Asian','Children Present','Completed College','Completed Graduate School','HOH Age 19 - 25','HOH Age 26 - 35',
 'HOH Age 36 - 45','HOH Age 46 - 55','HOH Age 56 - 65','HOH Age 66+','High School Graduates or Below','Hispanic','Income $100,000 - $124,999','Income $125,000 - $149,999',
 'Income $15,000 - $24,999','Income $150,000 - $174,999','Income $175,000 - $199,999','Income $200,000 - $249,999','Income $25,000 - $34,999','Income $250,000+',
 'Income $35,000 - $49,999','Income $50,000 - $74,999','Income $75,000 - $99,999','Income <$15,000','Married','No Children Present','Not Married',
 'Other Ethnicity','Some College','Unknown Ethnicity','Unknown Income','White'], axis=1)

In [5]:
hh_df_member.reset_index(inplace=True)
hh_df_member.head()

Value,MOSAIC_SEGMENT_ID,1 Member HH,2 Member HH,3 Member HH,4 Member HH,5+ Member HH
0,A01,186658.0,592834.0,361185.0,327471.0,512606.0
1,A02,192502.0,307736.0,188898.0,136836.0,134943.0
2,A03,1093.0,121875.0,129444.0,139132.0,295750.0
3,A04,10041.0,119219.0,141673.0,173179.0,406588.0
4,A05,152312.0,639755.0,155378.0,52111.0,35809.0


In [6]:
mk_df_new = mk_df.drop(['Unnamed: 0','Prob'],axis = 1)

In [7]:
mk_df_new_renamed=mk_df_new.rename(columns={"Mosaic":"MOSAIC_SEGMENT_ID"})
mk_df_new_renamed.head()

,MOSAIC_SEGMENT_ID,Custom_seg
0,A06,2
1,K39,2
2,A05,2
3,J35,3
4,L43,3


In [8]:
custom_seg=mk_df_new_renamed.sort_values('MOSAIC_SEGMENT_ID',ascending=True)
custom_seg.reset_index(inplace=True)
custom_seg=custom_seg.drop(['index','MOSAIC_SEGMENT_ID'],axis=1)
custom_seg


,Custom_seg
0,2
1,2
2,4
3,2
4,2
5,2
6,2
7,6
8,4
9,2


In [9]:
total_df=hh_df_member.join(custom_seg)

In [10]:
total_df.head()

,MOSAIC_SEGMENT_ID,1 Member HH,2 Member HH,3 Member HH,4 Member HH,5+ Member HH,Custom_seg
0,A01,186658.0,592834.0,361185.0,327471.0,512606.0,2
1,A02,192502.0,307736.0,188898.0,136836.0,134943.0,2
2,A03,1093.0,121875.0,129444.0,139132.0,295750.0,4
3,A04,10041.0,119219.0,141673.0,173179.0,406588.0,2
4,A05,152312.0,639755.0,155378.0,52111.0,35809.0,2


In [11]:
Seg_total=total_df.groupby("Custom_seg")["1 Member HH","2 Member HH","3 Member HH","4 Member HH","5+ Member HH"].sum()
print(Seg_total)

            1 Member HH  2 Member HH  3 Member HH  4 Member HH  5+ Member HH
Custom_seg                                                                  
1             5887704.0    7264630.0    1769406.0    1022800.0     1268260.0
2             9221257.0    9817557.0    4456676.0    2945954.0     3682544.0
3             9835545.0   10629709.0    4854753.0    2541915.0     2685733.0
4             2018836.0    6066565.0    2567031.0    2212400.0     3832285.0
5             7217600.0    9903760.0    2016488.0    1130369.0     1887247.0
6             2005983.0    1536395.0     586522.0     467795.0     1186311.0


In [12]:
# custom_seg=Seg_total_1.sum(axis=1)
Seg_total['Segment_HH_Count']=Seg_total.sum(axis=1)
Seg_total

,1 Member HH,2 Member HH,3 Member HH,4 Member HH,5+ Member HH,Segment_HH_Count
Custom_seg,,,,,,
1,5887704.0,7264630.0,1769406.0,1022800.0,1268260.0,17212800.0
2,9221257.0,9817557.0,4456676.0,2945954.0,3682544.0,30123988.0
3,9835545.0,10629709.0,4854753.0,2541915.0,2685733.0,30547655.0
4,2018836.0,6066565.0,2567031.0,2212400.0,3832285.0,16697117.0
5,7217600.0,9903760.0,2016488.0,1130369.0,1887247.0,22155464.0
6,2005983.0,1536395.0,586522.0,467795.0,1186311.0,5783006.0


In [13]:
Seg_total['Total_HH']=Seg_total['Segment_HH_Count'].sum(axis=0)
Seg_total

,1 Member HH,2 Member HH,3 Member HH,4 Member HH,5+ Member HH,Segment_HH_Count,Total_HH
Custom_seg,,,,,,,
1,5887704.0,7264630.0,1769406.0,1022800.0,1268260.0,17212800.0,122520030.0
2,9221257.0,9817557.0,4456676.0,2945954.0,3682544.0,30123988.0,122520030.0
3,9835545.0,10629709.0,4854753.0,2541915.0,2685733.0,30547655.0,122520030.0
4,2018836.0,6066565.0,2567031.0,2212400.0,3832285.0,16697117.0,122520030.0
5,7217600.0,9903760.0,2016488.0,1130369.0,1887247.0,22155464.0,122520030.0
6,2005983.0,1536395.0,586522.0,467795.0,1186311.0,5783006.0,122520030.0


In [14]:
for column in Seg_total:
    Seg_total[f'Perc_{column}']=(Seg_total[column]/Seg_total['Segment_HH_Count'])*100
    
Seg_total

,1 Member HH,2 Member HH,3 Member HH,4 Member HH,5+ Member HH,Segment_HH_Count,Total_HH,Perc_1 Member HH,Perc_2 Member HH,Perc_3 Member HH,Perc_4 Member HH,Perc_5+ Member HH,Perc_Segment_HH_Count,Perc_Total_HH
Custom_seg,,,,,,,,,,,,,,
1,5887704.0,7264630.0,1769406.0,1022800.0,1268260.0,17212800.0,122520030.0,34.205382,42.204813,10.279594,5.942090,7.368121,100.0,711.796047
2,9221257.0,9817557.0,4456676.0,2945954.0,3682544.0,30123988.0,122520030.0,30.611010,32.590496,14.794442,9.779429,12.224623,100.0,406.719157
3,9835545.0,10629709.0,4854753.0,2541915.0,2685733.0,30547655.0,122520030.0,32.197381,34.797136,15.892392,8.321146,8.791945,100.0,401.078348
4,2018836.0,6066565.0,2567031.0,2212400.0,3832285.0,16697117.0,122520030.0,12.090926,36.333009,15.374097,13.250192,22.951777,100.0,733.779550
5,7217600.0,9903760.0,2016488.0,1130369.0,1887247.0,22155464.0,122520030.0,32.577065,44.701208,9.101538,5.101987,8.518201,100.0,553.001418
6,2005983.0,1536395.0,586522.0,467795.0,1186311.0,5783006.0,122520030.0,34.687548,26.567411,10.142165,8.089132,20.513743,100.0,2118.621872


In [15]:
Seg_total=Seg_total.drop(['1 Member HH','2 Member HH','3 Member HH','4 Member HH','5+ Member HH','Segment_HH_Count','Total_HH','Perc_Segment_HH_Count','Perc_Total_HH'],axis=1)
Seg_total.reset_index(inplace=True)
Seg_total

,Custom_seg,Perc_1 Member HH,Perc_2 Member HH,Perc_3 Member HH,Perc_4 Member HH,Perc_5+ Member HH
0,1,34.205382,42.204813,10.279594,5.942090,7.368121
1,2,30.611010,32.590496,14.794442,9.779429,12.224623
2,3,32.197381,34.797136,15.892392,8.321146,8.791945
3,4,12.090926,36.333009,15.374097,13.250192,22.951777
4,5,32.577065,44.701208,9.101538,5.101987,8.518201
5,6,34.687548,26.567411,10.142165,8.089132,20.513743


In [16]:
panel_df.head()

,Custom_Seg,Perc_1 Member HH,Perc_2 Member HH,Perc_3 Member HH,Perc_4 Member HH,Perc_5+ Member HH
0,1,20,20,20,20,20
1,2,19,36,5,10,30
2,3,7,15,46,25,7
3,4,12,10,36,35,7
4,5,15,7,20,30,28


In [17]:
# Seg_total.values

In [18]:
# Seg_total.values[0]

In [19]:
ary1=scipy.spatial.distance.euclidean(Seg_total.values[0],panel_df.values[0])
ary2=scipy.spatial.distance.euclidean(Seg_total.values[1],panel_df.values[1])
ary3=scipy.spatial.distance.euclidean(Seg_total.values[2],panel_df.values[2])
ary4=scipy.spatial.distance.euclidean(Seg_total.values[3],panel_df.values[3])
ary5=scipy.spatial.distance.euclidean(Seg_total.values[4],panel_df.values[4])
ary6=scipy.spatial.distance.euclidean(Seg_total.values[5],panel_df.values[5])

In [20]:
ary1+ary2+ary3+ary4+ary5+ary6

223.37182733197423